In [101]:
# importing required libraries

import os
import tensorflow as tf
import datetime
from tensorflow import keras
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
#%load_ext tensorboard
import seaborn as sns
import Augmentor
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

### Question-1

### Part a):- Performing EDA of the given dataset

In [33]:
df = pd.read_csv(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\train.csv', index_col = 0)

In [34]:
df.head()

,healthy,multiple_diseases,rust,scab
image_id,,,,
Train_0,0,0,0,1
Train_1,0,1,0,0
Train_2,1,0,0,0
Train_3,0,0,1,0
Train_4,1,0,0,0


In [35]:
df.shape

(1821, 4)

In [36]:
#checking for null values

df.isnull().sum()

healthy              0
multiple_diseases    0
rust                 0
scab                 0
dtype: int64

In [109]:
from sklearn.model_selection import train_test_split

traindf, cvdf = train_test_split(df, test_size=0.2, random_state=42)

print(traindf.shape)
print(cvdf.shape)

(1456, 4)
(365, 4)


In [118]:
traindf

,healthy,multiple_diseases,rust,scab
image_id,,,,
Train_1455,0,0,1,0
Train_1552,1,0,0,0
Train_300,1,0,0,0
Train_1708,0,1,0,0
Train_611,0,0,1,0
...,...,...,...,...
Train_1130,0,0,0,1
Train_1294,0,0,0,1
Train_860,0,0,1,0


In [38]:
import shutil
from shutil import copyfile

# delete temp dir
if os.path.exists(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp'):
    shutil.rmtree(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp')

os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp')

# train directory
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\train')
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\train\healthy')
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\train\multiple_diseases')
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\train\rust')
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\train\scab')

# validation directory
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\valid')
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\valid\healthy')
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\valid\multiple_diseases')
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\valid\rust')
os.mkdir(r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\valid\scab')

In [119]:
SOURCE = 'C:/Users/conno/Downloads/work/1. Study(main objective)/Artificial Intelligence/4. Intellipaat AI course\Modules/3. AI(DEEP LEARNING)/Assignments/Live-class/Assignment-6/dataset/images/'

TRAIN_DIR = r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\train'

for index, data in traindf.iterrows():
    label = df.columns[np.argmax(data)]
    filepath = os.path.join(SOURCE, index + ".jpg") # Adding index to images
    destination = os.path.join(TRAIN_DIR, label, index + ".jpg") # filtering the images by type and adding to the destination folder 
    copyfile(filepath, destination)
    
for subdir in os.listdir(TRAIN_DIR): # using a for loop to iterate trough all the subdirectories and printing their respective lengths..
    print(subdir, len(os.listdir(os.path.join(TRAIN_DIR, subdir))))

In [120]:
VALID_DIR = r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\valid'

# copy images to valid directory
for index, data in testdf.iterrows():
    label = df.columns[np.argmax(data)]
    filepath = os.path.join(SOURCE, index + ".jpg")
    destination = os.path.join(VALID_DIR, label, index + ".jpg")
    copyfile(filepath, destination)
    
for subdir in os.listdir(VALID_DIR):
    print(subdir, len(os.listdir(os.path.join(VALID_DIR, subdir))))

In [45]:
#train_output = train_generator.next() # everytime this is called, it will produce a next step of images.

In [46]:
# images = train_output[0]
# labels = train_output[1]

In [47]:
# images.shape 

### Prepairing the dataset...

In [ ]:
#from skimage.io import imread_collection
#images = imread_collection(r"C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\kaggle\temp\train\healthy\*.JPG", conserve_memory=True)

In [138]:
Train_dir = r'C:\Users\conno\Downloads\work\1. Study(main objective)\Artificial Intelligence\4. Intellipaat AI course\Modules\3. AI(DEEP LEARNING)\Assignments\Live-class\Assignment-6\dataset\images\train'

SyntaxError: EOL while scanning string literal (231162228.py, line 1)

### Performing image Augmentation...

In [ ]:
training_datagen = ImageDataGenerator(rescale = 1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(TRAIN_DIR, target_size=(150,150), class_mode='categorical', batch_size=32)
validation_generator = validation_datagen.flow_from_directory(VALID_DIR, target_size=(150,150), class_mode='categorical', batch_size=32)

### Model-Building

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])


model.compile(loss = 'categorical_crossentropy', 
              optimizer = Adam(0.0001),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [1]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

history=model.fit(train_generator,
                       steps_per_epoch = STEP_SIZE_TRAIN,  # say 319 images per epoch/ to complete an epoch and no of epochs is 5
                      validation_data = valid_generator,
                      epochs = 12,
                    validation_steps= STEP_SIZE_VALID,
                     callbacks=[tensorboard_callback])

### Analyzing metrics

In [ ]:


train_loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
train_accuracy = model.history.history['accuracy']
validation_accuracy = model.history.history['val_accuracy']

In [ ]:
plt.plot(train_loss)
plt.plot(val_loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("LOSS CURVE")
plt.legend(['Train', 'Validation'])
plt.grid()
plt.show()

In [ ]:
plt.plot(train_accuracy)
plt.plot(validation_accuracy)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("ACCURACY CURVE")
plt.legend(['Train', 'Validation'])
plt.grid()
plt.show()

### References:

https://github.com/mdbloice/Augmentor


https://www.kaggle.com/code/geochatz/plant-pathology-classification-with-tensorflow/notebook